In [144]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import numpy as np

In [145]:
#Pull data from csv file- traffic camera locations
csv_file = "traffic-cameras-1.csv"
traffic_cameras_df = pd.read_csv(csv_file)
traffic_cameras_df["Longitude"] = traffic_cameras_df["Longitude"].map("{:.2f}".format)
traffic_cameras_df["Latitude"] = traffic_cameras_df["Latitude"].map("{:.2f}".format)
traffic_cameras_df.head()

,Camera ID,Camera Manufacturer,Turn on Date,ATD Location ID,Primary St Segment ID,Cross St Segment ID,Landmark,Signal Engineer Area,Council District,JURISDICTION,...,CROSS_ST_AKA,Cross Street Block,PRIMARY_ST_AKA,Location,IP Comm Status,IP Comm Status Date and Time,Longitude,Latitude,Published Screenshots,Screenshot Address
0,68,Sarix,NaN,LOC16-000180,2017636.0,2017637.0,NaN,CENTRAL,9,NaN,...,NaN,401.0,NaN,"(30.2837963, -97.7419052)",OFFLINE,06/23/2017 01:59:00 AM +0000,-97.74,30.28,NaN,http://162.89.4.145/CCTVImages/CCTV68.jpg
1,1,Sarix,11/18/2016,LOC16-005090,3268115.0,NaN,"Little Walnut Creek Library, HEB",NORTHEAST,4,NaN,...,NaN,NaN,NaN,"(30.3636856, -97.6981583)",OFFLINE,10/02/2017 05:18:00 PM +0000,-97.70,30.36,NaN,http://162.89.4.145/CCTVImages/CCTV1.jpg
2,3,Sarix,12/20/2016,LOC16-004915,5102830.0,2029964.0,NaN,NORTHWEST,6,NaN,...,NaN,10001.0,NaN,"(30.4819565, -97.7895737)",OFFLINE,05/11/2017 05:00:00 AM +0000,-97.79,30.48,NaN,http://162.89.4.145/CCTVImages/CCTV3.jpg
3,4,Spectra Enhanced,11/30/2016,LOC16-003630,2040576.0,2020020.0,NaN,CENTRAL,10,NaN,...,NaN,3400.0,NaN,"(30.31354, -97.76716)",OFFLINE,05/11/2017 05:00:00 AM +0000,-97.77,30.31,NaN,http://162.89.4.145/CCTVImages/CCTV4.jpg
4,2,Spectra Enhanced,11/08/2016,LOC16-000805,2018548.0,2018299.0,NaN,NORTHEAST,"3,1",NaN,...,NaN,3300.0,NaN,"(30.2735558, -97.6979065)",ONLINE,10/02/2017 08:46:00 PM +0000,-97.70,30.27,NaN,http://162.89.4.145/CCTVImages/CCTV2.jpg


In [146]:
cam_columns = ["Camera ID", "Turn on Date", "Location Name", "Longitude","Latitude"]
new_traffic_cameras_df = traffic_cameras_df[cam_columns].copy()
new_traffic_cameras_df = new_traffic_cameras_df.rename(columns={"Camera ID":"Camera_ID","Turn on Date":"Turn_on_Date","Location Name":"Location_Name"})
new_traffic_cameras_df = new_traffic_cameras_df.set_index(["Camera_ID"])
new_traffic_cameras_df.head()

,Turn_on_Date,Location_Name,Longitude,Latitude
Camera_ID,,,,
68,NaN,GUADALUPE ST / 21ST ST,-97.74,30.28
1,11/18/2016,830 BLK W RUNDBERG LN (Little Walnut Creek Lib...,-97.70,30.36
3,12/20/2016,STAKED PLAINS DR / LAKELINE BLVD,-97.79,30.48
4,11/30/2016,35TH ST / PECOS ST,-97.77,30.31
2,11/08/2016,AIRPORT BLVD / OAK SPRINGS DR,-97.70,30.27


In [147]:
new_traffic_cameras_df.dtypes

Turn_on_Date     object
Location_Name    object
Longitude        object
Latitude         object
dtype: object

In [148]:
#Pull data from csv file- traffic incidents
csv_file = "small_incident_report.csv"
traffic_incidents_df = pd.read_csv(csv_file, encoding="utf-8")
traffic_incidents_df

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status
0,066B5BB7A384BED2EDCDC65D0BCAA57C1EE971D0_15083...,10/18/2017 02:28:00 PM +0000,zSTALLED VEHICLE,"(30.261230,-97.736767)",30.261230,-97.736767,N Ih 35 Svrd Sb & E Cesar Chavez St,ARCHIVED
1,0A362CA04E5D1A1A33B972938C55AC2595D989F9_15083...,10/18/2017 01:41:00 PM +0000,Traffic Impediment,"(30.185598,-97.812579)",30.185598,-97.812579,1700-1930 W Dittmar Rd,ARCHIVED
2,0CE1D1D8CFA8BDADD7BC865D3E87BBF2FBBC003E_15083...,10/18/2017 02:53:00 PM +0000,COLLISION,"(30.285479,-97.812326)",30.285479,-97.812326,4310 Bee Caves Rd,ARCHIVED
3,0DE1F0466A1DDBB39B3249511CD3FE60BE0D4D6C_15083...,10/18/2017 10:51:00 AM +0000,Traffic Hazard,"(30.401971,-97.728792)",30.401971,-97.728792,0 N Mopac Nb At Mopac Railro Trn,ARCHIVED
4,12F5AA9F4574E1267551B2B784C315C53E19903D_15083...,10/18/2017 05:16:00 PM +0000,Crash Service,"(30.265218,-97.736408)",30.265218,-97.736408,700-717 E 5th St,ARCHIVED
5,1533AD56150B32637090BE48AF8EEF374BDE71E4_15083...,10/18/2017 01:51:00 PM +0000,Crash Urgent,"(30.352207,-97.723261)",30.352207,-97.723261,7806-7899 Woodrow Ave,ARCHIVED
6,178F71645D8D762522EDE90115F4F50F1E19D62B_15083...,10/18/2017 05:01:00 PM +0000,Traffic Hazard,"(30.361598,-97.743869)",30.361598,-97.743869,7900-7999 N MOPAC EXPY SVRD NB,ARCHIVED
7,185B44A1DEEF7AC2851ECBD3FA9B3CDD1A40C152_15083...,10/18/2017 02:48:00 PM +0000,Crash Service,"(30.348984,-97.676554)",30.348984,-97.676554,9210 Cameron Rd,ARCHIVED
8,18C775519E031F57D09D66471927ECBBA6D45826_15083...,10/18/2017 04:48:00 PM +0000,Crash Service,"(30.433105,-97.768612)",30.433105,-97.768612,Jollyville Rd & Barrington Way,ARCHIVED
9,18DD8025688A8458D55DABF67D5CE9D57E155B82_15083...,10/18/2017 12:46:00 PM +0000,Crash Service,"(30.367675,-97.613282)",30.367675,-97.613282,12239-12251 Harris Branch Pkwy,ARCHIVED


In [149]:
traffic_incidents_df.dtypes

Traffic Report ID     object
Published Date        object
Issue Reported        object
Location              object
Latitude             float64
Longitude            float64
Address               object
Status                object
dtype: object

In [150]:
traffic_incidents_df["Longitude"] = traffic_incidents_df["Longitude"].map("{:.2f}".format)
# traffic_incidents_df["Latitude"] = traffic_incidents_df["Latitude"].str.replace(',','').astype(np.float64)
traffic_incidents_df["Latitude"] = traffic_incidents_df["Latitude"].map("{:.2f}".format)
traffic_incidents_df.dropna()
traffic_incidents_df.head()

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status
0,066B5BB7A384BED2EDCDC65D0BCAA57C1EE971D0_15083...,10/18/2017 02:28:00 PM +0000,zSTALLED VEHICLE,"(30.261230,-97.736767)",30.26,-97.74,N Ih 35 Svrd Sb & E Cesar Chavez St,ARCHIVED
1,0A362CA04E5D1A1A33B972938C55AC2595D989F9_15083...,10/18/2017 01:41:00 PM +0000,Traffic Impediment,"(30.185598,-97.812579)",30.19,-97.81,1700-1930 W Dittmar Rd,ARCHIVED
2,0CE1D1D8CFA8BDADD7BC865D3E87BBF2FBBC003E_15083...,10/18/2017 02:53:00 PM +0000,COLLISION,"(30.285479,-97.812326)",30.29,-97.81,4310 Bee Caves Rd,ARCHIVED
3,0DE1F0466A1DDBB39B3249511CD3FE60BE0D4D6C_15083...,10/18/2017 10:51:00 AM +0000,Traffic Hazard,"(30.401971,-97.728792)",30.40,-97.73,0 N Mopac Nb At Mopac Railro Trn,ARCHIVED
4,12F5AA9F4574E1267551B2B784C315C53E19903D_15083...,10/18/2017 05:16:00 PM +0000,Crash Service,"(30.265218,-97.736408)",30.27,-97.74,700-717 E 5th St,ARCHIVED


In [151]:
incident_columns = ["Traffic Report ID","Published Date", "Issue Reported", "Longitude", "Latitude", "Address"]
new_traffic_incidents_df = traffic_incidents_df[incident_columns].copy()
new_traffic_incidents_df.dropna()
new_traffic_incidents_df = new_traffic_incidents_df.rename(columns={"Traffic Report ID":"Traffic_Report_ID","Published Date":"Published_Date","Issue Reported":"Issue_Reported"})
new_traffic_incidents_df = new_traffic_incidents_df.set_index(["Traffic_Report_ID"])
new_traffic_incidents_df.head()

,Published_Date,Issue_Reported,Longitude,Latitude,Address
Traffic_Report_ID,,,,,
066B5BB7A384BED2EDCDC65D0BCAA57C1EE971D0_1508318881000,10/18/2017 02:28:00 PM +0000,zSTALLED VEHICLE,-97.74,30.26,N Ih 35 Svrd Sb & E Cesar Chavez St
0A362CA04E5D1A1A33B972938C55AC2595D989F9_1508316087000,10/18/2017 01:41:00 PM +0000,Traffic Impediment,-97.81,30.19,1700-1930 W Dittmar Rd
0CE1D1D8CFA8BDADD7BC865D3E87BBF2FBBC003E_1508320416000,10/18/2017 02:53:00 PM +0000,COLLISION,-97.81,30.29,4310 Bee Caves Rd
0DE1F0466A1DDBB39B3249511CD3FE60BE0D4D6C_1508305877000,10/18/2017 10:51:00 AM +0000,Traffic Hazard,-97.73,30.40,0 N Mopac Nb At Mopac Railro Trn
12F5AA9F4574E1267551B2B784C315C53E19903D_1508328964000,10/18/2017 05:16:00 PM +0000,Crash Service,-97.74,30.27,700-717 E 5th St


In [152]:
new_traffic_incidents_df.dtypes

Published_Date    object
Issue_Reported    object
Longitude         object
Latitude          object
Address           object
dtype: object

In [153]:
connection_string = "root:Miamiheat5@localhost/ETL_db"
engine = create_engine(f'mysql://{connection_string}')

In [154]:
engine.table_names()

['traffic_cameras', 'traffic_incidents']

In [155]:
new_traffic_cameras_df.to_sql(name='traffic_cameras', con=engine, if_exists='append', index=True)
new_traffic_incidents_df.to_sql(name='traffic_incidents', con=engine, if_exists='append', index=True)

In [ ]:
merged_df = 